<a href="https://colab.research.google.com/github/IKKEM-Lin/colab/blob/main/path_search_20230918.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 产物路径搜索，需上传[ttl文件](https://github.com/IKKEM-Lin/colab/blob/main/gen_turtle_20230901.ipynb)

In [1]:
# install dependencies
! pip install rdflib
! pip install requests
! pip install loguru
!pip install pyvis
!pip install igraph
import igraph as ig

from pyvis.network import Network

from rdflib import Namespace, Literal, URIRef, Graph
import requests
from loguru import logger


from collections import deque
import copy
import json
import uuid
import collections
import hashlib
import os
import re

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 14.6 MB/s eta 0:00:00


### 1. 公共函数，用于统一化合物的名称，https://pubchem.deno.dev 会缓存所请求的数据

In [2]:
# common function

def get_IUPAC_name_final(name, mapping_dict):
    try:
        r = requests.get(f"https://pubchem.deno.dev/iupac?name={name}")
        result = json.loads(r.text)
        if result.get("data"):
            return result.get("data") or name
    except:
        return name

def get_spieces_class_operations(key, mapping_dict, IUPAC_name = ""):
    def get_md5(name):
        return hashlib.md5(name.encode("UTF-8")).hexdigest()

    def get_CHEBI_ID(name):
        try:
            r = requests.get(f"https://pubchem.deno.dev/chebi?name={name}")
            result = json.loads(r.text)
            if result.get("data"):
                return result.get("data") or name
        except:
            return name

    if not IUPAC_name:
        IUPAC_name = get_IUPAC_name_final(key, mapping_dict)
    name_list = IUPAC_name.split(";")
    name = name_list[0] if name_list else ""
    tag = get_CHEBI_ID(name)
    if "CHEBI" in tag:
        re_tag = ''.join(re.findall("(CHEBI_\d+)", tag))
        return re_tag, URIRef("obo:" + re_tag)
    else:
        id_str = get_md5(IUPAC_name)
        return id_str, URIRef("spi:"+"{}".format(id_str))


def get_spieces(name):
  # 拿到name对应的ID
  id_str, URI = get_spieces_class_operations(name, {})
  if "CHEBI" in id_str:
    return URIRef("obo:{}".format(id_str))
  else:
    return URIRef("spi:{}".format(id_str))

### 2. 读取ttl文件

In [3]:
rg = Graph()
rg.parse("./10000.ttl", format='turtle')

sparql_prefix =  "SELECT * WHERE {{ \n" \
            "\t{expression}\n"\
          "}}\n"

def is_reaction(node):
  return "react:" in str(node)

def get_substance_name(substance):
  obo = substance
  if is_reaction(obo):
    return substance
  if not isinstance(substance, URIRef):
    obo = URIRef(obo)
  query = sparql_prefix.format(expression = f"""
    <{obo}> <spi:has_IUPAC_name> ?IUPAC_name.
    OPTIONAL {{<{obo}> <spi:has_name> ?name}}
    OPTIONAL {{<{obo}> <spi:has_formula> ?formula}}
  """) + "LIMIT 5"
  result = rg.query(query)
  result = list(map(lambda x: str(x.formula or x.name), result))
  return result and result[0] or substance

### 3. 构造子图并整形化（如果已有igraph_data.json可跳过）

In [14]:
node_map = [] # 节点映射
ig_edges = [] # 边

#  图裁剪
triples1 = rg.triples((None, URIRef("react:has_product"), None))
triples2 = rg.triples((None, URIRef("spi:is_reactant_of"), None))
triples3 = rg.triples((None, URIRef("spi:is_product_of"), None))

rg2 = Graph()
rg2 += triples1
rg2 += triples2

# 仅需has_product及is_reactant_of即可构建所需映射，如果搜索过程中需使用其他变量，此次需调整
for s,p,o in rg2:
  s_index = -1
  if s in node_map:
    s_index = node_map.index(s)
  else:
    node_map.append(s)
    s_index = len(node_map) - 1

  o_index = -1
  if o in node_map:
    o_index = node_map.index(o)
  else:
    node_map.append(o)
    o_index = len(node_map) - 1
  ig_edges.append((s_index, o_index))

rg2 += triples3
print(len(rg2), len(node_map), len(ig_edges))

with open("igraph_data.json", "w") as f:
  json.dump([node_map, ig_edges], f)

32505 9957 22351


### 4. 读取整形数据并构造搜索函数

In [15]:
with open("igraph_data.json", "r") as f:
  node_map, ig_edges = json.load(f)
g = ig.Graph(n=len(node_map), edges=ig_edges, directed=True)
g.summary()

substance_only_product = ["CO", "H2", "syngas", "H2O", "CO2", "Carbon Dioxide", "Carbon Monoxide"]
substance_only_product = tuple([str(get_spieces(x)) in node_map and node_map.index(str(get_spieces(x))) for x in substance_only_product])

# 反应聚类
reaction_group = {}
def get_all_reaction():
  return [node_map.index(x) for x in node_map if "react:" in str(x)]
def get_reaction_uniq_key(graph, reaction):
  reactants = sorted(map(lambda x: str(x), graph.neighbors(reaction, mode="in")))
  products = sorted(map(lambda x: str(x), graph.neighbors(reaction, mode="out")))
  key = ",".join(reactants) + ";" + ",".join(products)
  return key
for reaction in get_all_reaction():
  key = get_reaction_uniq_key(g, reaction)
  if reaction_group.get(key):
    reaction_group.get(key).append(reaction)
  else:
    reaction_group[key] = [reaction]
print("聚合后的种类：", len(reaction_group.keys()), "所有的反应数量", len(get_all_reaction()))

def find_loops_within_n_steps(name, n_steps, graph=g):
  query_str = str(get_spieces(name))
  if query_str not in node_map:
    return [[query_str]]
  vertex_id = node_map.index(query_str)

  duplicated_reactions = set()
  queue = deque([(vertex_id, 0, [vertex_id])])  # The queue will contain tuples of (vertex, depth, path).
  loops = []
  while queue:
    current_vertex, depth, path = queue.popleft()
    if depth == n_steps or current_vertex in substance_only_product:
      loops.append(path)
      continue

    neighborhood = list(graph.neighbors(current_vertex, mode="in"))

    # todo: 按照之前算法，如果反应接下来没有反应物，则跳过该路径。此处需再考量
    if len(neighborhood)==0 and depth % 2 == 1:
      continue

    # todo: 如果是接下来任一可能的反应物出现在前面的路径中，则不再扩展。此处需再考量
    substance_in_previous = False
    temp_path = [vertex_id] + path
    if depth % 2 == 1:
      substance_in_previous = any([item in temp_path for item in neighborhood])
      # print(vertex_id, substance_in_previous)

    neighborhood_not_in_path = [item for item in neighborhood if (item not in temp_path and item not in duplicated_reactions)]
    if substance_in_previous or (not neighborhood_not_in_path):
      loops.append(path)
      continue
    for neighbor in neighborhood_not_in_path:
      if neighbor in duplicated_reactions:
        continue
      if depth % 2 == 0:
        key = get_reaction_uniq_key(g, neighbor)
        for x in reaction_group.get(key, [neighbor]):
          duplicated_reactions.add(x)
      new_path = path + [neighbor]
      queue.append((neighbor, depth + 1, new_path))
  return loops

聚合后的种类： 3254 所有的反应数量 7930


### 5. 测试并打印结果

In [17]:
res = find_loops_within_n_steps("Butane", 7)
paths = [[node_map[item] for item in path] for path in res]

print("----------------- Result ------------------------", len(paths))
result = {}
for path in paths:
  temp = result
  for key in path[:-1]:
    if key not in temp:
      temp[key] = {}
    temp = temp[key]

  last_key = path[-1]
  if last_key not in temp:
    temp[last_key] = {}

def print_dict(d, indent=0):
    for key, value in d.items():
        print('  ' * indent + str(get_substance_name(key)))
        if isinstance(value, dict):
            print_dict(value, indent+2)
# print_dict(result)
# with open("path7.json", "w") as f:
#   json.dump(paths, f)

----------------- Result ------------------------ 1587


### 6. 可视化构建

In [11]:
output_file = "tree.html"
option = {
    "hierarchical": False,
    "substance_same_node": False
}

data = result
net = Network(height="90vh", width="100%", notebook=True, layout= option.get("hierarchical") and {"direction": "LR"} or None )
colors = ["#CC9900", "#999900", "#669900", "#339900", "009900"]

def add_node(net, node_id, label, parent=None, level = 0):
    node_color = parent and "#0099CC" or "#992211"
    if level % 2 == 1:
      node_color = colors[level // 2]
    net.add_node(node_id, label=label, color=node_color, size=parent and 25 or 40)
    if parent:
        net.add_edge(parent, node_id)

def build_tree(net, data, parent=None, level = 0):
    count = 0
    for key, value in data.items():
        count += 1
        node_id = option.get("substance_same_node") and key or ((parent or "") + key + str(count))
        add_node(net, node_id, get_substance_name(key), parent, level)
        if isinstance(value, dict):
            new_level = level + 1
            build_tree(net, value, parent=node_id, level = new_level)

build_tree(net, data)
net.show(output_file)

with open(output_file) as f:
    content = f.read()

content = re.sub(r'<center>|</center>', '', content)

# 下面操作将层次图由上下结构改为左右，注释可恢复，可参考：https://ame.cool/pages/84ec1c/#%E9%85%8D%E7%BD%AE%E9%A1%B9%E8%AF%A6%E6%83%85
if option.get("hierarchical"):
  content = re.sub(r'"hierarchical": {', '"hierarchical": { direction: "LR",', content)

with open(output_file, 'w') as f:
    f.write(content)


# 上传展示
import IPython
upload_url = "https://reaction-tree.deno.dev"
files = {'file': (output_file, open(output_file, 'rb'), 'text/html', {'Expires': '0'})}
file_url = requests.post(upload_url, files = files).json().get("Key")
# print(file_url)
if file_url:
  html = f"<strong>展示地址：</strong><a target='_blank' href='{upload_url}/{file_url}'>{upload_url}/{file_url}</a>"
IPython.display.HTML(html)


tree.html
